In [ ]:

import pandas as pd
import numpy
from sklearn.model_selection import train_test_split
import os
import gc

import matplotlib.pyplot as plt 
import imageio
import PIL
from PIL import ImageFile
import cv2
import numpy as np
from IPython.display import display
from tqdm import tqdm
from imblearn.over_sampling import RandomOverSampler

import math
import albumentations
import random
from tqdm import tqdm
import tensorflow as tf

ImageFile.LOAD_TRUNCATED_IMAGES = True
%matplotlib inline
     

In [ ]:
def Scaler(array):
    return np.log(array + 0.01)

def invScaler(array):
    return np.exp(array - 0.01)

def pad_to_shape(array, from_shape=900, to_shape=928, how='mirror'):
    padding = int( (to_shape - from_shape)/2 )
    if how == 'zero':
        array_padded = np.pad(array, ((0,0), (padding,padding), (padding, padding), (0,0)), mode = 'constant', constant_values = 0)
    elif how == 'mirror':
        array_padded = np.pad(array, ((0,0), (padding,padding), (padding,padding), (0,0)), mode = 'reflect')
    return array_padded

def pred_to_rad(pred, from_shape=928, to_shape=900):
    padding = int((from_shape - to_shape)/2)
    return pred[::, padding:padding+to_shape, padding:padding+to_shape].copy()

In [ ]:
def data_preprocessing(X):
    X = np.moveaxis(X, 0, -1)
    X = X[np.newaxis, ::, ::, ::]
    X = Scaler(X)
    X = pad_to_shape(X)
    return X

def data_postprocessing(nwcst):
    nwcst = pred_to_rad(nwcst)
    nwcst = invScaler(nwcst)
    nwcst = np.where(nwcst>0, nwcst, 0)
    return nwcst

In [ ]:
class Dataset(tf.keras.utils.Sequence):
    def __init__(
        self,
        dataset_dict,
        image_names,
        batch_size
    ):
        self.keys = image_names
        self.dataset = dataset_dict
        self.bs = batch_size

    def get_index(self, i):
        x = []
        for j in range(4):
            try:
                arr = np.array(self.dataset.get(self.keys[i+j]))
            except:
                print(i, j)
            x.append(arr)

        x = data_preprocessing(x)
        x = np.squeeze(x)
        y = np.array(self.dataset[self.keys[i+4]])[np.newaxis, ::, ::]
        y = data_preprocessing(y)
        y = np.squeeze(y)

        return x.astype('float32'), y.astype('float32')


    def __getitem__(self, index):
        X = []
        Y = []
        for i in range (self.bs*index, self.bs*(index+1)):
            x, y = self.get_index(i)
            X.append(x[np.newaxis, :])
            Y.append(y[np.newaxis, :])
        return X, Y

    def __len__(self):
        return (len(self.keys)-4)//self.bs
        

In [ ]:
import h5py
dataset_dict = h5py.File('/kaggle/input/rydl-rainnet/RYDL.hdf5', 'r')
print(dataset_dict)

In [ ]:
image_names_full = list(dataset_dict.keys())

In [ ]:
image_names = [name for name in tqdm(image_names_full) if name[0:4]>"2012"]
train_images = image_names[:int(total_images*0.8)]
val_images = image_names[int(total_images*0.8):int(total_images*0.9)]
test_images = image_names[int(total_images*0.9):]
print(len(train_images), len(val_images), len(test_images))

In [ ]:
arr = np.array(dataset_dict.get(image_names[0]))
print(arr.shape)
plt.figure(figsize=(5,5))
plt.imshow(arr)

In [ ]:
train_dataset = Dataset(
    dataset_dict = dataset_dict,
    image_names = train_images,
    batch_size = 1
)

val_dataset = Dataset(
    dataset_dict = dataset_dict,
    image_names = val_images,
    batch_size = 1
)

test_dataset = Dataset(
    dataset_dict = dataset_dict,
    image_names = test_images,
    batch_size = 1
)

In [ ]:
X_train = []
Y_train = []
for i in range(train_dataset.__len__()):
    X, Y = train_dataset.__getitem__(i)
    X = np.squeeze(np.array(X))
    Y = np.squeeze(np.array(Y))
    X_train.append(X)
    Y_train.append(Y)

del train_dataset
gc.collect()

X_val = []
Y_val = []
for i in range(val_dataset.__len__()):
    X, Y = val_dataset.__getitem__(i)
    X = np.squeeze(np.array(X))
    Y = np.squeeze(np.array(Y))
    X_val.append(X)
    Y_val.append(Y)

del val_dataset
gc.collect()


X_test = []
Y_test = []
for i in range(test_dataset.__len__()):
    X, Y = test_dataset.__getitem__(i)
    X = np.squeeze(np.array(X))
    Y = np.squeeze(np.array(Y))
    X_test.append(X)
    Y_test.append(Y)

del test_dataset
gc.collect()

In [ ]:
print(type(X_train), X_train[0].shape)
print(type(X_val), X_val[0].shape)

In [ ]:
def create_dataset(X_data, Y_data):
    # Tạo tf.data.Dataset từ các mảng numpy
    dataset = tf.data.Dataset.from_tensor_slices((X_data, Y_data))
    
    dataset = dataset.shuffle(buffer_size = 10000)  
    dataset = dataset.batch(1)  # Định nghĩa kích thước batch
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)  # Tối ưu hóa việc tải dữ liệu
    
    return dataset

In [ ]:
train_set = create_dataset(X_train, Y_train)
del X_train, Y_train
gc.collect()

val_set = create_dataset(X_val, Y_val)
del X_val, Y_val
gc.collect()

test_set = create_dataset(X_test, Y_test)
# del X_test, Y_test
# gc.collect()

In [ ]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

def rainnet(input_shape=(928, 928, 4), mode="regression"):

    inputs = Input(input_shape)

    conv1f = Conv2D(64, 3, padding='same', kernel_initializer='he_normal')(inputs)
    conv1f = Activation("relu")(conv1f)
    conv1s = Conv2D(64, 3, padding='same', kernel_initializer='he_normal')(conv1f)
    conv1s = Activation("relu")(conv1s)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1s)

    conv2f = Conv2D(128, 3, padding='same', kernel_initializer='he_normal')(pool1)
    conv2f = Activation("relu")(conv2f)
    conv2s = Conv2D(128, 3, padding='same', kernel_initializer='he_normal')(conv2f)
    conv2s = Activation("relu")(conv2s)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2s)

    conv3f = Conv2D(256, 3, padding='same', kernel_initializer='he_normal')(pool2)
    conv3f = Activation("relu")(conv3f)
    conv3s = Conv2D(256, 3, padding='same', kernel_initializer='he_normal')(conv3f)
    conv3s = Activation("relu")(conv3s)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3s)

    conv4f = Conv2D(512, 3, padding='same', kernel_initializer='he_normal')(pool3)
    conv4f = Activation("relu")(conv4f)
    conv4s = Conv2D(512, 3, padding='same', kernel_initializer='he_normal')(conv4f)
    conv4s = Activation("relu")(conv4s)
    drop4 = Dropout(0.5)(conv4s)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5f = Conv2D(1024, 3, padding='same', kernel_initializer='he_normal')(pool4)
    conv5f = Activation("relu")(conv5f)
    conv5s = Conv2D(1024, 3, padding='same', kernel_initializer='he_normal')(conv5f)
    conv5s = Activation("relu")(conv5s)
    drop5 = Dropout(0.5)(conv5s)

    up6 = concatenate([UpSampling2D(size=(2, 2))(drop5), conv4s], axis=3)
    conv6 = Conv2D(512, 3, padding='same', kernel_initializer='he_normal')(up6)
    conv6 = Activation("relu")(conv6)
    conv6 = Conv2D(512, 3, padding='same', kernel_initializer='he_normal')(conv6)
    conv6 = Activation("relu")(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3s], axis=3)
    conv7 = Conv2D(256, 3, padding='same', kernel_initializer='he_normal')(up7)
    conv7 = Activation("relu")(conv7)
    conv7 = Conv2D(256, 3, padding='same', kernel_initializer='he_normal')(conv7)
    conv7 = Activation("relu")(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2s], axis=3)
    conv8 = Conv2D(128, 3, padding='same', kernel_initializer='he_normal')(up8)
    conv8 = Activation("relu")(conv8)
    conv8 = Conv2D(128, 3, padding='same', kernel_initializer='he_normal')(conv8)
    conv8 = Activation("relu")(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1s], axis=3)
    conv9 = Conv2D(64, 3, padding='same', kernel_initializer='he_normal')(up9)
    conv9 = Activation("relu")(conv9)
    conv9 = Conv2D(64, 3, padding='same', kernel_initializer='he_normal')(conv9)
    conv9 = Activation("relu")(conv9)
    conv9 = Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    
    if mode == "regression":
        outputs = Conv2D(1, 1, activation='linear')(conv9)
    elif mode == "segmentation":
        outputs = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=outputs)

    return model

In [ ]:
from tensorflow.keras import backend as K
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

def r2_score(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))

model = rainnet()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),loss='log_cosh', metrics=['mse', rmse, r2_score])

In [ ]:
checkpoint_filepath = '/kaggle/working/ckpt/checkpoint.weights.h5'

my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=4)
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, save_best_only=True),
]


history = model.fit(train_set, epochs = 40, validation_data = val_set, callbacks=my_callbacks)

In [ ]:
import matplotlib.pyplot as plt

# Vẽ đồ thị mất mát qua các epochs
plt.plot(history.history['loss'], label='Training loss')
if 'val_loss' in history.history:
    plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Loss Progression')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
results = model.evaluate(test_set)

In [ ]:
y_true = data_postprocessing(np.array(Y_test))
y_pred = data_postprocessing(np.array(model.predict(test_set)))

import matplotlib.pyplot as plt

num_images = 3

plt.figure(figsize=(20,10))
for i in range(num_images):
    # Hiển thị ảnh thực tế
    ax = plt.subplot(2, num_images, i + 1)
    plt.imshow(y_true[i].squeeze())  
    plt.title("Actual")
    plt.axis("off")
    
    # Hiển thị ảnh dự đoán
    ax = plt.subplot(2, num_images, i + 1 + num_images)
    plt.imshow(y_pred[i].squeeze())  
    plt.title("Predicted")
    plt.axis("off")

plt.tight_layout()
plt.show()